In [1]:
server = 'irc.chat.twitch.tv'
port = 6667
nickname = '' #username goes here
token = '' #oauth token goes here (include the oauth:)
channel = '#lord_kebun'
chat = 'lord_kebun_chat.log'

# Connection to Twitch IRC

To establish a connection to Twitch IRC we'll be using Python's socket library. First we need to instantiate a socket:

In [7]:
import socket

sock = socket.socket()

Next we'll connect this socket to Twitch by calling connect() with the server and port we defined above:

In [8]:
sock.connect((server, port))

Once connected, we need to send our token and nickname for authentication, and the channel to connect to over the socket.  
  

With sockets, we need to send() these parameters as encoded strings:

In [9]:
sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

17

PASS carries our token, NICK carries our username, and JOIN carries the channel. These terms are actually common among many IRC connections, not just Twitch. So you should be able to use this for other IRC you wish to connect to, but with different values.

Note that we send encoded strings by calling .encode('utf-8'). This encodes the string into bytes which allows it to be sent over the socket.

# Receiving channel messages

Now we have successfully connected and can receive responses from the channel we subscribed to. To get a single response we can call .recv() and then decode the message from bytes:

In [11]:
resp = sock.recv(2048).decode('utf-8')

resp #run this cell twice

':highstatus!highstatus@highstatus.tmi.twitch.tv PRIVMSG #lord_kebun :LANDED\r\n:silverswoosh!silverswoosh@silverswoosh.tmi.twitch.tv PRIVMSG #lord_kebun :whyyyyyyyyyyyyyyyyyy\r\n:bigslong88!bigslong88@bigslong88.tmi.twitch.tv PRIVMSG #lord_kebun :kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash kebunCrash\r\n:kinggod_zilla!kinggod_zilla@kinggod_zilla.tmi.twitch.tv PRIVMSG #lord_kebun :LUL\r\n:bugawhatt!bugawhatt@bugawhatt.tmi.twitch.tv PRIVMSG #lord_kebun :nailed it\r\n:zypher02!zypher02@zypher02.tmi.twitch.tv PRIVMSG #lord_kebun :smooth\r\n:s_a_u_c_e_y!s_a_u_c_e_y@s_a_u_c_e_y.tmi.twitch.tv PRIVMSG #lord_kebun :holy shit\r\n:retcon__!retcon__@retcon__.tmi.twitch.tv PRIVMSG #lord_kebun :11/10\r\n:dyam0900!dyam0900@dyam0900.tmi.twitch.tv PRIVMSG #lord_kebun :Booooooooom\r\n:gabzo1!gabzo1@gabzo1.tmi.twitch.tv PRIVMSG #lord_kebun :LUL\r\n:toombstonehc!toombstonehc@toombstonehc.tmi.twitch.tv PRIVMSG #lord_kebun :nailed

Note: running this the first time will show a welcome message from Twitch. Run it again to show the first message from the channel.

Example output (if you were to connect to Ninja's stream):
                
```
':spappygram!spappygram@spappygram.tmi.twitch.tv PRIVMSG #ninja :Chat, let Ninja play solos if he wants. His friends can get in contact with him.\r\n'
```

The 2048 is the buffer size in bytes, or the amount of data to receive. The convention is to use small powers of 2, so 1024, 2048, 4096, etc. Rerunning the above will receive the next message that was pushed to the socket.

If we need to close and/or reopen the socket just use:
    
    sock.close()

# Writing messages to a file
Right now, our socket is being inundated with responses from Twitch but we have two problems:

We need to continuously check for new messages
We want to log the messages as they come in
To fix, we'll use a loop to check for new messages while the socket is open and use Python's logging library to log messages to a file.

First, let's set up a basic logger in Python that will write messages to a file:

In [12]:
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler(chat, encoding='utf-8')])

We're setting the log level to DEBUG, which allows all levels of logging to be written to the file. The format is how we want each line to look, which will be the time we recorded the line and message from the channel separated by an em dash. The datefmt is how we want the time portion of the format to be recorded (example below).

Finally, we pass a FileHandler to handlers. We could give it multiple handlers to, for example we could add another handler that prints messages to the console. In this case, we're logging to chat.log, which will be created by the handler. Since we're passing a plain filename without a path, the handler will create this file in the current directory. Later on we'll make this filename dynamic to create separate logs for different channels.

Let's log the response we received earlier to test it out:

In [13]:
logging.info(resp)

You can check your log wherever you saved the chat.log file

So we have the time the message was logged at the beginning, a double dash separator, and then the message. This format corresponds to the format argument we used in basicConfig.

Later, we'll be parsing these each message and use the time as a piece of data to explore.

# Continuous message writing
Now on to continuously checking for new messages in a loop.

When we're connected to the socket, Twitch (and other IRC) will periodically send a keyword — "PING" — to check if you're still using the connection. We want to check for this keyword, and send an appropriate response — "PONG".

One other thing we'll do is parse emojis so they can be written to a file. To do this, we'll use the emoji library that will provide a mapping from emojis to their meaning in words. For example, if a 👍 shows up in a message it'll be converted to :thumbs_up:.

The following is a while loop that will continuously check for new messages from the socket, send a PONG if necessary, and log messages with parsed emojis:



In [14]:
from emoji import demojize

while True:
    resp = sock.recv(2048).decode('utf-8')

    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))
        
#you have to manually end this one or else it keeps running

KeyboardInterrupt: 

This will keep running until you stop it. To see the messages in real-time open a new terminal, navigate to the log's location, and run:

tail -f chat.log

# Parsing logs
Our goal for this section is to parse the chat log into a pandas DataFrame to prepare for analysis

The columns we'd like to have for analysis are:

date and time
sender's username
and the message
We'll need to parse the information from each line, so let's look at an example line again:

In [15]:
msg = '2019-05-17_01:46:54 — :doctajay28!doctajay28@doctajay28.tmi.twitch.tv PRIVMSG #lord_kebun :kebunCG kebunCG'

In [16]:
from datetime import datetime

time_logged = msg.split()[0].strip()

time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

time_logged

datetime.datetime(2019, 5, 17, 1, 46, 54)

Great! We have a datetime. Let's parse the rest of the message.

Since using an em dash (—, or Right-ALT+0151 on Windows) is sometimes used in chat, we will need to split on it, skip the date, and rejoin with an em dash to ensure the message is the same:



In [17]:
username_message = msg.split('—')[1:]
username_message = '—'.join(username_message).strip()

username_message

':doctajay28!doctajay28@doctajay28.tmi.twitch.tv PRIVMSG #lord_kebun :kebunCG kebunCG'

The message is structure with a username at the beginning, a '#' denoting the channel, and a colon to say where the message begins.

Regex is great for this kind of thing. We have three pieces of info we want to extract from a well-formatted string.

In the regex search below, each parentheses — (.*) — will capture that part of the string:

In [18]:
import re

username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message).groups()

print(f"Channel: {channel} \nUsername: {username} \nMessage: {message}")

Channel: lord_kebun 
Username: doctajay28 
Message: kebunCG kebunCG


Excellent. Now we have each piece parsed. Let's loop through the entire chat log, parse each line like the example line, and create a DataFrame at the end. If you haven't used DataFrames that much, definitely check out our beginners guide to pandas.

Here's it all put together:

In [19]:
import pandas as pd

def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                username_message = '—'.join(username_message).strip()

                username, channel, message = re.search(
                    ':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', username_message
                ).groups()

                d = {
                    'dt': time_logged,
                    'channel': channel,
                    'username': username,
                    'message': message
                }

                data.append(d)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)
        
    
df = get_chat_dataframe(chat)

Let's quickly view what we have now:

In [20]:
df.set_index('dt', inplace=True)

print(df.shape)


(3343, 3)


In [21]:
pd.set_option('display.max_colwidth', -1)

In [22]:
df.tail(20)

,channel,message,username
dt,,,
2019-05-17 01:49:31,lord_kebun,!followage,oanmz
2019-05-17 01:49:31,lord_kebun,"@oanmz, oanmz has been following lord_kebun for 1 month 22 days 6 hours",streamelements
2019-05-17 01:49:36,lord_kebun,no host?,kappakopter
2019-05-17 01:49:37,lord_kebun,ACTION @lord_kebun stayhyBottle You've been live for just over 8 hours. By this point in your broadcast you should have consumed at least 32oz (960mL) of water to maintain optimum hydration.,stay_hydrated_bot
2019-05-17 01:49:37,lord_kebun,there you go,wakz156
2019-05-17 01:49:48,lord_kebun,Did he end?,gangster0823
2019-05-17 01:50:07,lord_kebun,joined just in time (:,grandmapeggy
2019-05-17 01:50:32,lord_kebun,Did Chang just stop streaming,chinesekush
2019-05-17 01:50:41,lord_kebun,go to sal boys,doanotherrunnn


In [23]:
import pickle
#put dataframe into a pickle for later use
pickle.dump(df,open("lord_kebun_chat.p","wb"))

Just from streaming messages over a couple of hours we have over 10,000 rows in our DataFrame.

Here's a few basic questions I'm particularly interested in:

Which user commented the most during this time period?
Which commands — words that start with ! — were used the most?
What are the most used emotes and emojis?
We'll use this dataset in the next article to explore these questions and more.

# From here
We've create a basic script to monitor a single channel on Twitch and successfully parsed the data into a DataFrame.

There's still many improvements that can be made. For example:

Variable logging file for monitoring and comparing different channels
Use Twitch API to retrieve live and popular channels under certain games
Generalize script to stream chat from multiple channels at once
Plus we need to answer those questions mentioned above and more.

In [24]:
sock.close() #closes the connection